In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Google Colab

In [2]:
%cd /content/drive/MyDrive/4_ING_INFORMATICA/tfg/tfg/experiments/taming_transformers/taming-transformers

/content/drive/MyDrive/4_ING_INFORMATICA/tfg/tfg/experiments/taming_transformers/taming-transformers


NGPU server

In [1]:
%cd /mnt/homeGPU1/pbedmar/pycharm/experiments/taming_transformers/taming-transformers/

/mnt/homeGPU1/pbedmar/pycharm/experiments/taming_transformers/taming-transformers


### Preprocesando imágenes de moléculas organometálicas de la Universidad de Neguev (articles_molecules)

In [2]:
import os
import glob
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps

In [4]:
def resize_with_padding(img, required_size):
  img.thumbnail((required_size, required_size))
  delta_w = required_size - img.size[0]
  delta_h = required_size - img.size[1]
  pad_w = delta_w // 2
  pad_h = delta_h // 2
  padding = (pad_w, pad_h, delta_w - pad_w, delta_h - pad_h)
  return ImageOps.expand(img, padding, fill=(255,255,255))

In [6]:
images_path = "../../../datasets/negev/articles_molecules/"
filenames = glob.glob(images_path+"*.png")

os.makedirs(images_path+"preprocess/", exist_ok=True)

required_size = 192
for filename in filenames:
  img = Image.open(filename)
  width, height = img.size
  img = resize_with_padding(img, required_size)
  
  img.convert("RGB").save(images_path+"preprocess/"+filename.split("/")[-1].replace("png","jpg"))

In [7]:
preprop_images_path = images_path+"preprocess/"
filenames = glob.glob(preprop_images_path+"*.jpg")
train, test = train_test_split(filenames, test_size=0.25, random_state=42)

In [8]:
with open(preprop_images_path+"xx_train.txt", "w") as f:
  for filename in train[:-1]:
    f.write(filename+"\n")
  f.write(train[-1])

with open(preprop_images_path+"xx_test.txt", "w") as f:
  for filename in test[:-1]:
    f.write(filename+"\n")
  f.write(test[-1])

### Aumentando dataset

In [34]:
import imgaug as ia
from imgaug import augmenters as iaa

augmentation_seq = iaa.Sequential([
                      iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),
                      iaa.Affine(
                          scale={"x": (0.8, 1.0), "y": (0.8, 1.0)},
                          rotate=(-25, 25),
                          shear=(-5,5),
                          cval=255
                      )
], random_order=True)


augmentation_seq_2 = iaa.Sequential([
                      iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),
                      iaa.LinearContrast((0.75, 1.5)),
                      iaa.Affine(
                          scale={"x": (0.7, 1.0), "y": (0.7, 1.0)},
                          rotate=(-45, 45),
                          shear=(-10,10),
                          translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                          cval=255
                      ),
                      iaa.Multiply((0.8, 1.2), per_channel=0.25),
                      iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.3),
                      iaa.Fliplr(0.3)
], random_order=True)

augmentation_seq_3 = iaa.Sequential([
                      iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),
                      iaa.LinearContrast((0.7, 1.75)),
                      iaa.Affine(
                          scale={"x": (0.7, 1.0), "y": (0.7, 1.0)},
                          rotate=(-45, 45),
                          shear=(-10,10),
                          translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                          cval=255
                      ),
                      iaa.Sometimes(0.7, iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.025*255))),
                      iaa.Fliplr(0.3)
], random_order=True)

In [32]:
import os
import glob
from sklearn.model_selection import train_test_split
import cv2

def aug(directory, aug_seq, repeat=2):
  ia.seed(1)
  images_path = "../../../datasets/negev/articles_molecules/"
  preprop_images_path = images_path+"preprocess/"
  filenames = glob.glob(preprop_images_path+"*.jpg")

  images = []
  for filename in filenames:
    img = cv2.imread(filename)
    images.append(img)

  augmented_images = images + aug_seq(images=images*repeat)

  os.makedirs(preprop_images_path+directory, exist_ok=True)
  count = 0
  for img in augmented_images:
    cv2.imwrite(preprop_images_path+directory+"/"+str(count)+".jpg", img)
    count += 1

def gen_metadata(directory):
  images_path = "../../../datasets/negev/articles_molecules/"
  preprop_images_path = images_path+"preprocess/"
  preprop_aug_images_path = preprop_images_path+directory+"/"
  filenames = glob.glob(preprop_aug_images_path+"*.jpg")
  train, test = train_test_split(filenames, test_size=0.25, random_state=42)

  with open(preprop_aug_images_path+"xx_train.txt", "w") as f:
    for filename in train[:-1]:
      f.write(filename+"\n")
    f.write(train[-1])

  with open(preprop_aug_images_path+"xx_test.txt", "w") as f:
    for filename in test[:-1]:
      f.write(filename+"\n")
    f.write(test[-1])

In [4]:
aug("aug", augmentation_seq, repeat=2)
gen_metadata("aug")

In [5]:
aug("aug2", augmentation_seq_2, repeat=3)
gen_metadata("aug2")

In [35]:
aug("aug3", augmentation_seq_3, repeat=3)
gen_metadata("aug3")